# Picky file parsing example

In [25]:
from probe import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Single file 

In [20]:
# set input 
input_file = './test-files/input-example-1.picky'
source_index = 1

In [32]:
%%time
# parse single file 
df = parse_picky_new(input_file, source_index)
print(df.shape)
df.head()

(1250, 13)
CPU times: user 3.24 s, sys: 16.7 ms, total: 3.26 s
Wall time: 3.26 s


,Annotation,Start,End,TargetSequence,Tm,ReverseComplement,Primer,Primer Tm,Spacer,Padlock,Padlock Tm,DeltaTm,source_index
0,NG_000006.1,26327,26368,CTGGGGTCCTTGAGTCGGACGGGCGTTTGTGCGTCTCCCGGC,91.22,GCCGGGAGACGCACAAACGCCCGTCCGACTCAAGGACCCCAG,GCCGGGAGACGCACAAACG,75.422692,CC,CGTCCGACTCAAGGACCCCAG,75.262572,0.160120,1
1,NG_000006.1,41924,41968,AGTTCCTGAGCCACGTTATCTCGGCGCTGGTTTCCGAGTACCGCT,89.22,AGCGGTACTCGGAAACCAGCGCCGAGATAACGTGGCTCAGGAACT,AGCGGTACTCGGAAACCAGC,73.668260,GC,CGAGATAACGTGGCTCAGGAACT,72.453816,1.214443,1
2,NG_007066.1,4920,4959,ACTTTGCACCAGTCCGAGGGAATTTGCGGTCGGTGACGCG,87.69,CGCGTCACCGACCGCAAATTCCCTCGGACTGGTGCAAAGT,CGCGTCACCGACCGCAAAT,75.143061,T,CCCTCGGACTGGTGCAAAGT,73.865102,1.277960,1
3,NG_007082.2,83837,83877,CTCCGGGGACATTACAAGCTGGTCGCCGTCATATTGGTAGC,83.78,GCTACCAATATGACGGCGACCAGCTTGTAATGTCCCCGGAG,GCTACCAATATGACGGCGAC,69.692239,C,AGCTTGTAATGTCCCCGGAG,71.024630,1.332391,1
4,NG_007082.2,86707,86748,TCCTTCTCCACGGCCTTGTATGGCGAGAGCGACCTGTGAGGC,88.59,GCCTCACAGGTCGCTCTCGCCATACAAGGCCGTGGAGAAGGA,GCCTCACAGGTCGCTCTCG,73.909610,CC,ATACAAGGCCGTGGAGAAGGA,72.335556,1.574054,1


## Multiple file 

In [26]:
# set input
input_path = './test-files/'
files = sorted([ f for f in os.listdir(input_path) if '.picky' in f])
print(files)

['input-example-1.picky', 'input-example-2.picky']


In [31]:
%%time
# setup multiple proessing for parsing the files 
input_file_1 = os.path.join(input_path, files[0])
input_file_2 = os.path.join(input_path, files[1])

p = [(input_file_1, 1), (input_file_2, 2)] # (input_file_path, source_index)
pool = mp.Pool(processes = (mp.cpu_count()))
results = pool.starmap(parse_picky_new, p)
pool.close()
pool.join()

df_combined = pd.concat(results)
print(df_combined.shape)
df_combined.head()

(2500, 13)
CPU times: user 38.8 ms, sys: 104 ms, total: 143 ms
Wall time: 4.36 s


,Annotation,Start,End,TargetSequence,Tm,ReverseComplement,Primer,Primer Tm,Spacer,Padlock,Padlock Tm,DeltaTm,source_index
0,NG_000006.1,26327,26368,CTGGGGTCCTTGAGTCGGACGGGCGTTTGTGCGTCTCCCGGC,91.22,GCCGGGAGACGCACAAACGCCCGTCCGACTCAAGGACCCCAG,GCCGGGAGACGCACAAACG,75.422692,CC,CGTCCGACTCAAGGACCCCAG,75.262572,0.160120,1
1,NG_000006.1,41924,41968,AGTTCCTGAGCCACGTTATCTCGGCGCTGGTTTCCGAGTACCGCT,89.22,AGCGGTACTCGGAAACCAGCGCCGAGATAACGTGGCTCAGGAACT,AGCGGTACTCGGAAACCAGC,73.668260,GC,CGAGATAACGTGGCTCAGGAACT,72.453816,1.214443,1
2,NG_007066.1,4920,4959,ACTTTGCACCAGTCCGAGGGAATTTGCGGTCGGTGACGCG,87.69,CGCGTCACCGACCGCAAATTCCCTCGGACTGGTGCAAAGT,CGCGTCACCGACCGCAAAT,75.143061,T,CCCTCGGACTGGTGCAAAGT,73.865102,1.277960,1
3,NG_007082.2,83837,83877,CTCCGGGGACATTACAAGCTGGTCGCCGTCATATTGGTAGC,83.78,GCTACCAATATGACGGCGACCAGCTTGTAATGTCCCCGGAG,GCTACCAATATGACGGCGAC,69.692239,C,AGCTTGTAATGTCCCCGGAG,71.024630,1.332391,1
4,NG_007082.2,86707,86748,TCCTTCTCCACGGCCTTGTATGGCGAGAGCGACCTGTGAGGC,88.59,GCCTCACAGGTCGCTCTCGCCATACAAGGCCGTGGAGAAGGA,GCCTCACAGGTCGCTCTCG,73.909610,CC,ATACAAGGCCGTGGAGAAGGA,72.335556,1.574054,1


## Filtering

In [ ]:
# TBA